In [2]:
from keras.layers import Dense, LSTM, GRU, Dropout, Flatten, Embedding,  BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keras
from keras.models import Model,Sequential
from keras.utils import to_categorical
from keras import regularizers
from keras.optimizers import SGD
from clr_callback import *
%matplotlib inline
import time
import pickle
import os
import math
import torch.nn as nn
from torch.nn.modules.utils import _triple
from keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report



In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
SPLIT_METHOD_child = "CHILD"
SPLIT_METHOD_session = "SESSION"
SPLIT_METHOD_random = "RANDOM"

# WITH FACE, BODY, HANDS (411) =>
FEAT_NUM_all = 201

# WITH BODY AND HANDS (201) =>
FEAT_NUM_body = (25 + 21 + 21) * 3

# WITH FACE ONLY (210) =>
FEAT_NUM_face = 70 * 3

SEQ_LENGTH = 1

NB_CLASS = 6
CSV_FILE = "ASD_36_res1.csv"
VARS_DIR = "var"


In [5]:
import pandas
data = pandas.read_csv('ASD_36_res1.csv')

In [6]:
X = data.iloc[:, 3:78]
y = data['engagement']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=1)

In [7]:
X_train.shape


(253124, 75)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [9]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(227811, 75)
(25313, 75)
(41207, 75)


In [10]:
X_train = X_train.to_numpy()
X_test =X_test.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_val = y_val.to_numpy()

In [11]:
type(X_train)

numpy.ndarray

In [12]:

early_stop = EarlyStopping(monitor='loss', patience=2)


In [13]:
xx = tf.convert_to_tensor(X_train.reshape(227811,75 , 1))
xv = tf.convert_to_tensor(X_val.reshape(25313, 75, 1))
xt = tf.convert_to_tensor(X_test.reshape(41207, 75, 1))
yy = tf.convert_to_tensor(y_train.reshape(227811, 1))
yv = tf.convert_to_tensor(y_val.reshape(25313, 1))
yt = tf.convert_to_tensor(y_test.reshape(41207, 1))

In [14]:
yy= to_categorical(yy)
yv =  to_categorical(yv)
yt = to_categorical(yt)

In [63]:
model = keras.Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(75,1)))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(Dropout(0.2))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(6, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_81 (Conv1D)           (None, 73, 64)            256       
_________________________________________________________________
conv1d_82 (Conv1D)           (None, 71, 64)            12352     
_________________________________________________________________
conv1d_83 (Conv1D)           (None, 69, 32)            6176      
_________________________________________________________________
conv1d_84 (Conv1D)           (None, 67, 32)            3104      
_________________________________________________________________
dropout_20 (Dropout)         (None, 67, 32)            0         
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 33, 32)            0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 1056)            

In [64]:
hist = model.fit(x = xx , y = yy, epochs=15, validation_data =(xv, yv), batch_size= 32)


Epoch 1/15
7120/7120 [==============================] - 25s 4ms/step - loss: 1.3817 - accuracy: 0.4679 - val_loss: 1.3300 - val_accuracy: 0.4836
Epoch 2/15
7120/7120 [==============================] - 26s 4ms/step - loss: 1.2591 - accuracy: 0.5111 - val_loss: 1.2198 - val_accuracy: 0.5273
Epoch 3/15
7120/7120 [==============================] - 26s 4ms/step - loss: 1.1871 - accuracy: 0.5411 - val_loss: 1.1573 - val_accuracy: 0.5543
Epoch 4/15
7120/7120 [==============================] - 25s 4ms/step - loss: 1.1379 - accuracy: 0.5611 - val_loss: 1.1309 - val_accuracy: 0.5668
Epoch 5/15
7120/7120 [==============================] - 24s 3ms/step - loss: 1.1003 - accuracy: 0.5776 - val_loss: 1.1061 - val_accuracy: 0.5770
Epoch 6/15
7120/7120 [==============================] - 25s 3ms/step - loss: 1.0696 - accuracy: 0.5905 - val_loss: 1.0790 - val_accuracy: 0.5884
Epoch 7/15
7120/7120 [==============================] - 24s 3ms/step - loss: 1.0450 - accuracy: 0.6004 - val_loss: 1.0552 - val_ac

In [65]:
h = model.evaluate(xt,yt)

1288/1288 [==============================] - 2s 2ms/step - loss: 0.9790 - accuracy: 0.6359
